In [1]:
import tensorflow as tf
import os
#os.environ["TF_GPU_THREAD"]="gpu_private"
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
from datetime import datetime
%load_ext tensorboard

print(tf.version.VERSION)


2.5.0


In [2]:
train_filepath_dataset = tf.data.TFRecordDataset.list_files("data/train/*.tfrecord", shuffle=True)
train_num_files = train_filepath_dataset.cardinality()
#num_files=30
train_dataset = train_filepath_dataset.interleave(
    lambda x: tf.data.TFRecordDataset(x),
    cycle_length=train_num_files,
    block_length=10,
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    deterministic=False)


In [3]:
validate_filepath_dataset = tf.data.TFRecordDataset.list_files("data/validate/*.tfrecord", shuffle=True)
validate_num_files = validate_filepath_dataset.cardinality()
#num_files=30
validate_dataset = validate_filepath_dataset.interleave(
    lambda x: tf.data.TFRecordDataset(x),
    cycle_length=validate_num_files,
    block_length=10,
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    deterministic=False)

In [4]:
context_features = {"population_size" : tf.io.FixedLenFeature((), tf.int64),
                   "selection_coefficient" : tf.io.FixedLenFeature((), tf.float32)}
sequence_features = {"raw_trait_frequencies" : tf.io.RaggedFeature(tf.float32)}

pop_size_min = 20
pop_size_max = 500
sc_min = 0.
sc_max = 0.5
min_survival=20
max_time_steps = 50 # to avoid OOM issues
num_trait_data = 100

def preprocess_and_split_into_tuples(tfrecord):
    # parse sequence example and normalise
    example = tf.io.parse_sequence_example(tfrecord, context_features=context_features, sequence_features=sequence_features)
    trait_frequencies = example[1]["raw_trait_frequencies"]
    pop_size_norm = (tf.cast(example[0]["population_size"], dtype=tf.float32) - pop_size_min) / (pop_size_max - pop_size_min)
    sc_norm = (tf.cast(example[0]["selection_coefficient"], dtype=tf.float32) - sc_min) / (sc_max - sc_min)
    # filter dataset to only include num_trait_data trials in which the trait survives at least min_survival
    trait_frequencies = tf.gather(trait_frequencies, tf.where(trait_frequencies.row_lengths() > min_survival), axis=0)
    
    trait_frequencies = trait_frequencies[0:num_trait_data, :, 0:max_time_steps]
    
    # cast labels into a list of tensors matching the features
    trait_frequencies = tf.squeeze(trait_frequencies, axis=1)
    label_tensors = tf.reshape(tf.stack((tf.repeat(pop_size_norm, num_trait_data), tf.repeat(sc_norm, num_trait_data)), axis=1), 
                               shape=(num_trait_data,2))
    # add final dimension (number features)
    label_tensors = tf.expand_dims(label_tensors, axis=2)
    trait_frequencies = tf.expand_dims(trait_frequencies, axis=2)

    return tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(trait_frequencies), tf.data.Dataset.from_tensor_slices(label_tensors)))


In [6]:
# test cycle length and block length for performance
train_dataset = train_dataset.interleave(
    preprocess_and_split_into_tuples,
    cycle_length=train_num_files,
    block_length=1,
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    deterministic=False)


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [7]:
# test cycle length and block length for performance
validate_dataset = validate_dataset.interleave(
    preprocess_and_split_into_tuples,
    cycle_length=validate_num_files,
    block_length=1,
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    deterministic=False)


In [8]:
batch_size = 24
train_dataset = train_dataset.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
validate_dataset = validate_dataset.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)


In [9]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(None, 1), ragged=True))
#model.add(tf.keras.layers.Input(shape=(None, 1000)))
model.add(tf.keras.layers.LSTM(50, activation='tanh'))
model.add(tf.keras.layers.Dense(2, activation='linear'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10400     
_________________________________________________________________
dense (Dense)                (None, 2)                 102       
Total params: 10,502
Trainable params: 10,502
Non-trainable params: 0
_________________________________________________________________


In [10]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(None, 1), ragged=True))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, activation='tanh', return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, activation='tanh')))
model.add(tf.keras.layers.Dense(100, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal()))
model.add(tf.keras.layers.Dense(2, activation='linear'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, None, 100)         20800     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 81,402
Trainable params: 81,402
Non-trainable params: 0
_________________________________________________________________


In [11]:
def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

def abs_dist(y_true, y_pred):
    abs_difference = tf.math.abs(y_true - y_pred)
    return tf.reduce_mean(abs_difference, axis=-1)

optimizer = tf.keras.optimizers.Adam(0.002)
model.compile(loss="mse", optimizer=optimizer, metrics=[abs_dist, tf.keras.metrics.RootMeanSquaredError()])
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1)



In [12]:
history = model.fit(train_dataset, epochs=10, validation_data=validate_dataset, callbacks=[callback, tensorboard_callback])


Epoch 1/10
1811/1811 [==============================] - 367s 200ms/step - loss: 0.0254 - abs_dist: 0.1206 - root_mean_squared_error: 0.1595 - val_loss: 0.0156 - val_abs_dist: 0.0943 - val_root_mean_squared_error: 0.1251
Epoch 2/10
 382/1811 [=====>........................] - ETA: 4:29 - loss: 0.0169 - abs_dist: 0.0982 - root_mean_squared_error: 0.1299

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir=logs

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(None, 1), ragged=True))
model.add(tf.keras.layers.LSTM(50, activation='tanh', return_sequences=True))
model.add(tf.keras.layers.LSTM(50, activation='tanh', return_sequences=True))
model.add(tf.keras.layers.LSTM(100, activation='tanh'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='linear'))
model.summary()

def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

def abs_dist(y_true, y_pred):
    abs_difference = tf.math.abs(y_true[0] - y_pred[0])
    return tf.reduce_mean(abs_difference, axis=-1)

optimizer = tf.keras.optimizers.Adam(0.002)
model.compile(loss="mse", optimizer=optimizer, metrics=[abs_dist, tf.keras.metrics.RootMeanSquaredError()])
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

history = model.fit(dataset, epochs=10, callbacks=[callback])


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(None, 1), ragged=True))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, activation='tanh', return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, activation='tanh')))
model.add(tf.keras.layers.Dense(100, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal()))
model.add(tf.keras.layers.Dense(2, activation='linear'))
model.summary()
def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

def abs_dist(y_true, y_pred):
    abs_difference = tf.math.abs(y_true - y_pred)
    return tf.reduce_mean(abs_difference, axis=-1)

optimizer = tf.keras.optimizers.Adam(0.0015)
model.compile(loss="mse", optimizer=optimizer, metrics=[abs_dist, tf.keras.metrics.RootMeanSquaredError()])
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

history = model.fit(dataset, epochs=20, callbacks=[callback])
